#Install gradio

In [1]:
pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.8 MB/s eta 0:00:00


#Import packages and data sets

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
data=pd.read_csv('/content/drive/MyDrive/MLP RECORD/dataset/bupa.csv')
data.head()

,mcv,alkphos,sgpt,sgot,gammagt,drinks,class
0,85,92,45,27,31,0.0,1
1,85,64,59,32,23,0.0,2
2,86,54,33,16,54,0.0,2
3,91,78,34,24,36,0.0,2
4,87,70,12,28,10,0.0,2


#preprocessing

In [4]:
data['class'] = data['class'].replace(1,0)
data['class'] = data['class'].replace(2,1)
data.head()

,mcv,alkphos,sgpt,sgot,gammagt,drinks,class
0,85,92,45,27,31,0.0,0
1,85,64,59,32,23,0.0,1
2,86,54,33,16,54,0.0,1
3,91,78,34,24,36,0.0,1
4,87,70,12,28,10,0.0,1


In [5]:
X=data.drop(columns=["class"])
y=data["class"]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
y_train

177    1
79     1
213    0
206    0
124    1
      ..
175    0
299    1
56     1
91     0
138    1
Name: class, Length: 276, dtype: int64

#Building Logistic Model

In [6]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6811594202898551

#Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred=dt.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6521739130434783

#Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6811594202898551

#XGBOOST

In [9]:
from xgboost import XGBClassifier
# fit model no training data
xg = XGBClassifier()
xg.fit(X_train, y_train)
y_pred=xg.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6666666666666666

#grid Search

In [16]:
from ast import mod
from sklearn.model_selection import GridSearchCV
parameters=[
            {'C':[0.25,0.5,0.75,1]}]
grid_search=GridSearchCV(estimator=lr,param_grid=parameters,scoring='accuracy',cv=10,n_jobs=1)
grid_search.fit(X_train,y_train)


GridSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=1,
             param_grid=[{'C': [0.25, 0.5, 0.75, 1]}], scoring='accuracy')

In [17]:
best_acc=grid_search.best_score_
best_par=grid_search.best_params_
print("best Accuracy:",best_acc*100)
print("best parameters:",best_par)


best Accuracy: 69.92063492063491
best parameters: {'C': 0.25}


**SVM model**

In [18]:
from sklearn.svm import SVC
svm=SVC(C=0.75,gamma=0.2,kernel='rbf')
svm.fit(X_train,y_train)
y_pred=svm.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6666666666666666

#Gradio input UI function

In [19]:
def prediction(model_selection,mcv,alkphos,sgpt,sgot,gammagt,drinks):
  new=pd.DataFrame([mcv,alkphos,sgpt,sgot,gammagt,drinks])
  new=new.transpose()
  new.columns=['mcv','alkphos','sgpt','sgot','gammagt','drinks']
  if model_selection=="SVC":
    pre=svm.predict(new)
  elif model_selection=="Logistic Regression":
    pre=lr.predict(new)
  elif model_selection=="Random Forest":
    pre=rf.predict(new)
  elif model_selection=="Desicion Tree" :
    pre=dt.predict(new)
  else :
    pre=xg.predict(new)

  if pre==0:
     return "The person has liver disorder"
  else:
     return " The person has no liver disorder"


#gradio Interface

In [20]:
import gradio as gr
interface = gr.Interface(fn=prediction, inputs=[
                                                gr.Radio(["SVC", "Logistic Regression", "Random Forest","Desicion Tree","XGBOOST"], label="Model selecton", info="Choose one of model to evaluate"),
                                                gr.Slider(70,100, value=80, label="mcv", info="Choose value"),
                                                gr.Slider(40,100, value=60, label="alkphos", info="Choose value"),
                                                gr.Slider(1,100, value=30, label="sgpt", info="Choose value"),
                                                gr.Slider(1, 100, value=30, label="sgot", info="Choose value"),
                                                gr.Slider(1, 100, value=30, label="gammagt", info="Choose value"),
                                                gr.Slider(0, 30, value=0.5, label="drinks", info="Choose value"),
                                                ],
                         outputs="text",
                         title="Liver Prediction",
                         live=True,

                        css="""body{background-color:darkblack} """,

                        )
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a4bd8b81da7529e19e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
